# Import Libraries

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

# Load data

In [3]:
df_assignment_rel=pd.read_csv('./data/assignment_relationships.csv')
df_training=pd.read_csv('./data/unit_test_scores.csv')
df_problem=pd.read_csv('./data/problem_details.csv')
df_actionlogs=pd.read_csv('./data/action_logs.csv')
df_problem=pd.read_csv('./data/problem_details.csv')

# Preprocess data (make data ready for analysis)

In [4]:
df_training=df_training.merge(df_assignment_rel,left_on='assignment_log_id',right_on='unit_test_assignment_log_id',how='left')
df_training=df_training[['unit_test_assignment_log_id','problem_id', 'score', 'in_unit_assignment_log_id']]
df_actionlogs=df_actionlogs[['assignment_log_id','problem_id','available_core_tutoring','action']]

one_hot = pd.get_dummies(df_actionlogs[['action','available_core_tutoring']])

# Concatenate the original DataFrame with the one-hot encoded DataFrame
df_actionlogs = pd.concat([df_actionlogs, one_hot], axis=1)
df_actionlogs=df_actionlogs.rename(columns={'problem_id':'problem_id_action'})

In [5]:
# Get all unique problems where skill_video or live_tutor available
problem_skill_video_available = df_actionlogs.loc[df_actionlogs['action_skill_related_video_requested'] == 1, 'problem_id_action']
problem_live_tutor_available = df_actionlogs.loc[df_actionlogs['action_live_tutor_requested'] == 1, 'problem_id_action']

problem_skill_video_available=problem_skill_video_available.to_list()
problem_live_tutor_available=problem_live_tutor_available.to_list()
problem_skill_video_available=list(set(problem_skill_video_available))
problem_live_tutor_available=list(set(problem_live_tutor_available))

In [6]:
df_training_hint=df_training.merge(df_actionlogs,left_on='in_unit_assignment_log_id',right_on='assignment_log_id',how='left')
# Create a new column with initial values as 0
df_training_hint['skill_related_video_available'] = 0
# Set values to 1 where the where problem is skill_video problem.
df_training_hint.loc[(df_training_hint['action'] == 'problem_started') & (df_training_hint['problem_id_action'].isin(problem_skill_video_available)), 'skill_related_video_available'] = 1

df_training_hint['live_tutor_available'] = 0
# Set values to 1 where the condition is satisfied
df_training_hint.loc[(df_training_hint['action'] == 'problem_started') & (df_training_hint['problem_id_action'].isin(problem_skill_video_available)), 'live_tutor_available'] = 1

# Tutoring analysis

In [7]:
# group data for each 'unit_test_assignment_log_id','problem_id'
df_training_hint_group=df_training_hint.groupby(['unit_test_assignment_log_id','problem_id'],as_index=False).agg({'problem_id_action':'nunique','available_core_tutoring_hint':'sum',
                                                                                                                  'action_answer_requested':'sum','action_correct_response':'sum','action_hint_requested':'sum',
                                                                                                                  'action_explanation_requested':'sum','action_live_tutor_requested':'sum',
                                                                                                                  'action_skill_related_video_requested':'sum','available_core_tutoring_answer':'sum',
                                                                                                                  'available_core_tutoring_explanation':'sum','available_core_tutoring_hint':'sum',
                                                                                                                  'action_correct_response':'sum','action_wrong_response':'sum',
                                                                                                                  'skill_related_video_available':'sum','live_tutor_available':'sum',
                                                                                                                  'score':'first'})

In [8]:
# calculate all tutoring type percentage requested and avilable
df_training_hint_group['per_hint_available']=df_training_hint_group['available_core_tutoring_hint']/df_training_hint_group['problem_id_action']
df_training_hint_group['per_explanation_available']=df_training_hint_group['available_core_tutoring_explanation']/df_training_hint_group['problem_id_action']
df_training_hint_group['per_answer_available']=df_training_hint_group['available_core_tutoring_answer']/df_training_hint_group['problem_id_action']
df_training_hint_group['per_skill_video_available']=df_training_hint_group['skill_related_video_available']/df_training_hint_group['problem_id_action']
df_training_hint_group['per_live_tutor_available']=df_training_hint_group['live_tutor_available']/df_training_hint_group['problem_id_action']
df_training_hint_group.loc[df_training_hint_group['per_hint_available']>1,'per_hint_available'] = 1
df_training_hint_group.loc[df_training_hint_group['per_explanation_available']>1,'per_explanation_available'] = 1
df_training_hint_group.loc[df_training_hint_group['per_answer_available']>1,'per_answer_available'] = 1
df_training_hint_group['per_hint_requested']=df_training_hint_group['action_hint_requested']/df_training_hint_group['available_core_tutoring_hint']
df_training_hint_group['per_explanation_requested']=df_training_hint_group['action_explanation_requested']/df_training_hint_group['available_core_tutoring_explanation']
df_training_hint_group['per_answer_requested']=df_training_hint_group['action_answer_requested']/df_training_hint_group['available_core_tutoring_answer']
df_training_hint_group['per_skill_video_requested']=df_training_hint_group['action_skill_related_video_requested']/df_training_hint_group['skill_related_video_available']
df_training_hint_group['per_live_tutor_requested']=df_training_hint_group['action_live_tutor_requested']/df_training_hint_group['live_tutor_available']
df_training_hint_group.loc[df_training_hint_group['per_hint_requested']>1,'per_hint_requested'] = 1
df_training_hint_group.loc[df_training_hint_group['per_explanation_requested']>1,'per_explanation_requested'] = 1
df_training_hint_group.loc[df_training_hint_group['per_answer_requested']>1,'per_answer_requested'] = 1
df_training_hint_group['total_available_toutoring']=df_training_hint_group['available_core_tutoring_hint']+df_training_hint_group['available_core_tutoring_explanation']+df_training_hint_group['available_core_tutoring_answer']+df_training_hint_group['action_skill_related_video_requested']+df_training_hint_group['action_live_tutor_requested']

In [9]:
df_training_hint_group['per_total_avai_tutoring']=df_training_hint_group['total_available_toutoring']/df_training_hint_group['problem_id_action']
df_training_hint_group['total_toutoring_requested']=df_training_hint_group['action_answer_requested']+df_training_hint_group['action_hint_requested']+df_training_hint_group['action_explanation_requested']+df_training_hint_group['action_skill_related_video_requested']+df_training_hint_group['action_live_tutor_requested']
df_training_hint_group['per_total_req_tutoring']=df_training_hint_group['total_toutoring_requested']/df_training_hint_group['problem_id_action']

df_training_hint_group.loc[df_training_hint_group['per_total_avai_tutoring']>1,'per_total_avai_tutoring'] = 1
df_training_hint_group.loc[df_training_hint_group['total_toutoring_requested']>1,'total_toutoring_requested'] = 1
df_training_hint_group['per_correct_response']=df_training_hint_group['action_correct_response']/df_training_hint_group['problem_id_action']
df_training_hint_group['per_wrong_response']=df_training_hint_group['action_wrong_response']/df_training_hint_group['problem_id_action']
df_training_hint_group.loc[df_training_hint_group['per_wrong_response']>1,'per_wrong_response'] = 1
df_training_hint_group.loc[df_training_hint_group['per_correct_response']>1,'per_correct_response'] = 1

# For grades separately

In [10]:
# get skill code for all problems.
df_training_hint_group=df_training_hint_group.merge(df_problem[['problem_id','problem_skill_code']],on='problem_id',how='left')
df_training_hint_group=df_training_hint_group.rename(columns={'problem_id_action':'number_of_problems'})
df_training_hint_group['grade_train'] = df_training_hint_group['problem_skill_code'].str.split('.', n=1).str[0]

In [11]:
upper_limit=0.7
lower_limit=0.3

In [12]:
# function to create dataframe of >=upper_limit and <lower limit with t-statistic and p-value
#input: upper_limit, lower_limit
#output: dict of dfs each for each tutoring type.

def calculate_metric_dfs(upper_limit, lower_limit):
    metrics = ['hint_requested', 'explanation_requested', 'answer_requested', 'total_req_tutoring', 'skill_video_requested', 'live_tutor_requested', 'correct_response', 'wrong_response']

    dfs = {}  # Dictionary to store the separate DataFrames

    for metric in metrics:
        df_metric = pd.DataFrame()  # Create an empty DataFrame for the metric
        for limit in [upper_limit, lower_limit]:
            if limit == upper_limit:
                limit_label = f'>={limit}'
                group = df_training_hint_group[df_training_hint_group[f'per_{metric}'] >= limit].groupby('grade_train')['score'].mean()
                total = df_training_hint_group[df_training_hint_group[f'per_{metric}'] >= limit]['score'].mean()
            else:
                limit_label = f'<{limit}'
                group = df_training_hint_group[df_training_hint_group[f'per_{metric}'] < limit].groupby('grade_train')['score'].mean()
                total = df_training_hint_group[df_training_hint_group[f'per_{metric}'] < limit]['score'].mean()
            group['Total'] = total
            column_name = f'{metric}({limit_label})'
            df_metric[column_name] = group

        # Perform statistical tests for each grade
        t_stats = []
        p_values = []
        for grade in df_metric.index:
            if grade == 'Total':
                # Perform statistical test for the total
                control_scores = df_training_hint_group[(df_training_hint_group[f'per_{metric}'] < lower_limit)]['score']
                treatment_scores = df_training_hint_group[(df_training_hint_group[f'per_{metric}'] >= upper_limit)]['score']
                t_stat, p_value = stats.ttest_ind(treatment_scores, control_scores)
                t_stats.append(t_stat)
                p_values.append(p_value)
            else:
                control_scores = df_training_hint_group[(df_training_hint_group[f'per_{metric}'] < lower_limit) & (df_training_hint_group['grade_train'] == grade)]['score']
                treatment_scores = df_training_hint_group[(df_training_hint_group[f'per_{metric}'] >= upper_limit) & (df_training_hint_group['grade_train'] == grade)]['score']
                t_stat, p_value = stats.ttest_ind(treatment_scores, control_scores)
                t_stats.append(t_stat)
                p_values.append(p_value)
                
        # Calculate score difference
        df_metric['score_difference'] = df_metric.iloc[:, 1] - df_metric.iloc[:, 0]
        df_metric[f'{metric}_t_statistic'] = t_stats
        df_metric[f'{metric}_p_value'] = p_values

        df_metric.rename_axis('Grade', inplace=True)

        dfs[metric] = df_metric  # Add the metric DataFrame to the dictionary

    return dfs


In [13]:

dfs_70_30 = calculate_metric_dfs(upper_limit, lower_limit)

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
